# Merge autoplus

In [1]:
import pandas as pd
import fuzzy_pandas as fpd
from fuzzywuzzy import fuzz
from fuzzywuzzy import process

In [2]:
autoplus = pd.read_csv('autoplus.csv')
autoplus = autoplus.drop(columns=['Unnamed: 0', 'Unnamed: 0.1', 'Unnamed: 0.1.1', 'Unnamed: 0.1.1.1', 'Unnamed: 0.1.1.1.1'])
display(autoplus[autoplus.duplicated()])
autoplus = autoplus.drop_duplicates()
autoplus['DATE'] = autoplus['DATE'].str.extract('(?P<Mois>\d{2})/(?P<Annee>\d{4})')['Annee']

,BOITE_VITESSE,DATE,ENERGIE,MALUS,MARQUE_MODELE,PRIX_NEUF,PUISS_FISCALE,VERSION
438,Manuelle,02/2020,Diesel,0,Dacia Lodgy,17850,6CV,Blue dCi 115 15 ans (Monospace)
507,Manuelle,02/2020,Diesel,0,Dacia Lodgy,18450,6CV,Blue dCi 115 15 ans (Monospace)
2846,Automatique,07/2019,Diesel,0,Nissan Qashqai,30500,6CV,1.5 dCi 115 DCT BVA Acenta (Tout-Terrain)
2936,Manuelle,07/2019,Essence,360,Nissan Qashqai,30800,8CV,1.3 DIG-T 160 N-Connecta (Tout-Terrain)
4252,Manuelle,12/2018,Diesel,1276,Nissan Qashqai,34800,8CV,1.7 dCi 150 Intelligent 4x4 Business Edition (...
4579,Manuelle,08/2019,Diesel,NaN,Fiat Talento,35628,5CV,2.0 MULTIJET 145 PACK PRO NAV (Fourgon)
4658,Manuelle,05/2019,Diesel,360,Nissan Qashqai,35850,8CV,1.7 dCi 150 Tekna (Tout-Terrain)
4989,Manuelle,08/2019,Diesel,NaN,Fiat Talento,36708,5CV,2.0 MULTIJET 170 PACK PRO NAV (Fourgon)
5037,Manuelle,08/2019,Diesel,NaN,Fiat Talento,36828,5CV,2.0 MULTIJET 145 PACK TECHNO (Fourgon)
5488,Manuelle,08/2019,Diesel,NaN,Fiat Talento,37908,5CV,2.0 MULTIJET 170 PACK TECHNO (Fourgon)


In [3]:
dcar = pd.read_csv('Data_cars.csv')
#dcar['right'] = dcar['Make'] + ' ' + dcar['Model'] + ' ' + dcar['Fuel']
dcar['Make_Model'] = dcar['Make'] + ' ' + dcar['Model']
dcar = dcar

In [4]:
display(autoplus.head(), dcar.head())

,BOITE_VITESSE,DATE,ENERGIE,MALUS,MARQUE_MODELE,PRIX_NEUF,PUISS_FISCALE,VERSION
0,Automatique,2013,Électrique,n.a.,Renault Twizy,7540,1CV,Electrique 45 BVA Life (2p.)
1,Manuelle,2019,Essence,100,Dacia Logan,8150,4CV,1.0 12V Sce 75 Access (4p.)
2,Automatique,2013,Électrique,n.a.,Renault Twizy,8240,2CV,Electrique 80 BVA Life (2p.)
3,Manuelle,2019,Essence,100,Dacia Sandero,8290,4CV,1.0i SCe 75 Access (5p.)
4,Automatique,2015,Électrique,n.a.,Renault Twizy,8340,1CV,Electrique 45 BVA Intens Blanc (2p.)


,Price,Make,Model,Model_year,Mileage,Fuel,Gearbox,Online,Description,Make_Model
0,11220.0,PEUGEOT,308,2014,94341.0 km,Diesel,mécanique,06/07/2018 à 3h47,"modele: 308 (2E GENERATION), version: 1.6 HDI ...",PEUGEOT 308
1,57526.0,BMW,X6,2015,39051.0 km,Diesel,automatique,03/04/2018 à 16h41,"modele: X6 F16, version: (F16) XDRIVE30D 258 M...",BMW X6
2,80379.0,AUDI,RS6,2014,75381.0 km,Essence,automatique,30/07/2018 à 1h55,"modele: RS6 (3E GENERATION) AVANT, version: II...",AUDI RS6
3,2830.0,FORD,KA,2007,92282.0 km,Essence,mécanique,09/07/2018 à 14h12,"modele: KA, version: 1.3 70 FUN, puissance_fis...",FORD KA
4,12925.0,FIAT,TIPO,2018,10.0 km,Essence,mécanique,19/05/2018 à 3h52,"modele: TIPO 2 SW, version: II SW 1.4 95 POP, ...",FIAT TIPO


In [17]:
matches = fpd.fuzzy_merge(autoplus, dcar,
                          left_on=['MARQUE_MODELE'],
                          keep_left=['BOITE_VITESSE', 'DATE', 'ENERGIE', 'MALUS', 'MARQUE_MODELE',
       'PRIX_NEUF', 'PUISS_FISCALE', 'VERSION'],
                          right_on=['Make_Model'],
                          keep_right=[dcar.columns],
                          ignore_case=True,
                          keep='match')

print(matches)

     MARQUE_MODELE      Make_Model
0    Dacia Sandero   DACIA SANDERO
1    Dacia Sandero   DACIA SANDERO
2    Dacia Sandero   DACIA SANDERO
3    Dacia Sandero   DACIA SANDERO
4    Dacia Sandero   DACIA SANDERO
5   Renault Twingo  RENAULT TWINGO
6   Renault Twingo  RENAULT TWINGO
7    Dacia Sandero   DACIA SANDERO
8    Dacia Sandero   DACIA SANDERO
9     Dacia Duster    DACIA DUSTER
10    Dacia Duster    DACIA DUSTER
11        Fiat 500        FIAT 500
12        Fiat 500        FIAT 500
13        Fiat 500        FIAT 500
14        Fiat 500        FIAT 500
15   Dacia Sandero   DACIA SANDERO
16   Dacia Sandero   DACIA SANDERO
17   Dacia Sandero   DACIA SANDERO
18   Dacia Sandero   DACIA SANDERO
19       Fiat Tipo       FIAT TIPO
20       Fiat Tipo       FIAT TIPO


In [18]:
def fuzzy_merge(df_1, df_2, key1, key2, threshold=90, limit=2):
    """
    df_1 is the left table to join
    df_2 is the right table to join
    key1 is the key column of the left table
    key2 is the key column of the right table
    threshold is how close the matches should be to return a match, based on Levenshtein distance
    limit is the amount of matches that will get returned, these are sorted high to low
    """
    s = df_2[key2].tolist()

    m = df_1[key1].apply(lambda x: process.extract(x, s, limit=limit))    
    df_1['matches'] = m

    m2 = df_1['matches'].apply(lambda x: ', '.join([i[0] for i in x if i[1] >= threshold]))
    df_1['matches'] = m2

    return df_1


In [22]:
fuzzy_merge(autoplus, dcar, 'MARQUE_MODELE', 'Make_Model', threshold=50).columns

Index(['BOITE_VITESSE', 'DATE', 'ENERGIE', 'MALUS', 'MARQUE_MODELE',
       'PRIX_NEUF', 'PUISS_FISCALE', 'VERSION', 'matches'],
      dtype='object')

In [5]:
from fuzzymatcher import link_table, fuzzy_left_join

# Columns to match on from df_left
left_on = ['Make_Model', 'Gearbox', 'Fuel']

# Columns to match on from df_right
right_on = ['MARQUE_MODELE', 'BOITE_VITESSE', 'ENERGIE']

# The link table potentially contains several matches for each record
#link_table(dcar, autoplus, left_on, right_on)


In [6]:
m = fuzzy_left_join(dcar, autoplus, left_on, right_on)
m[m['PRIX_NEUF'].isna()]


,best_match_score,__id_left,__id_right,Price,Make,Model,Model_year,Mileage,Fuel,Gearbox,...,Description,Make_Model,BOITE_VITESSE,DATE,ENERGIE,MALUS,MARQUE_MODELE,PRIX_NEUF,PUISS_FISCALE,VERSION


In [7]:
m['PRIX_NEUF'].isna().sum()

0

In [8]:
a = m.copy()
a.drop(['best_match_score', '__id_left', '__id_right'], axis = 1, inplace = True)
a.columns

Index(['Price', 'Make', 'Model', 'Model_year', 'Mileage', 'Fuel', 'Gearbox',
       'Online', 'Description', 'Make_Model', 'BOITE_VITESSE', 'DATE',
       'ENERGIE', 'MALUS', 'MARQUE_MODELE', 'PRIX_NEUF', 'PUISS_FISCALE',
       'VERSION'],
      dtype='object')

In [9]:
a.to_csv('mergedAutoplus.csv', index=False)